This is the final preprocessing notebook before modeling. Here, I'll start with trying out models' performances given various variable encoding strategies. I need to explore the possibility of feature reduction; I created multiple features in the previous notebook but am not yet sure if they'll be valuable in making predictions. Finally, it will be helpful to balance the data because currently the target variable 'reordered' is very unbalanced. I can begin to explore various models in the process of all this. 

Notebook on which this one builds: https://github.com/fractaldatalearning/Capstone2/blob/main/notebooks/preprocessing2_feature_engineering.ipynb

One thing to look out for in this notebook: If I'm modeling and the computer is doing fine processing the dataset at this size, I could go back to the notebook for preprocessing1, add more rows to further increments of the full original dataset, concatenate them, re-run all the feature engineering steps with the larger dataset, and come back here to try out modeling with more rows (from twice as many, perhaps up to 10 times as many).

In [4]:
import pandas as pd
import numpy as np
import os
from library.sb_utils import save_file

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

import random
import collections

from IPython.display import Audio
sound_file = './alert.wav'

In [3]:
df = pd.read_csv('../data/processed/features_engineered.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218232 entries, 0 to 218231
Data columns (total 26 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   order_id                218232 non-null  int64  
 1   user_id                 218232 non-null  int64  
 2   order_by_user_sequence  218232 non-null  int64  
 3   days_since_prior_order  218232 non-null  float64
 4   add_to_cart_sequence    218232 non-null  int64  
 5   reordered               218232 non-null  int64  
 6   product_name            218232 non-null  object 
 7   aisle_name              218232 non-null  object 
 8   dept_name               218232 non-null  object 
 9   prior_purchases         218232 non-null  float64
 10  free                    218232 non-null  int64  
 11  fresh                   218232 non-null  int64  
 12  mix                     218232 non-null  int64  
 13  natural                 218232 non-null  int64  
 14  organic             